In [3]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Bidirectional, TimeDistributed,Conv1D, MaxPooling1D, Input, concatenate
from keras.layers.recurrent import SimpleRNN
from keras.layers import GRU
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.callbacks import EarlyStopping
import os
import tarfile
import numpy as np
import re
import pandas as pd
import numpy

Using TensorFlow backend.
D:\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [42]:
def RNN(maxlen = 50, max_features = 4590, embed_size =32):
    model = Sequential()
    model.add(Embedding(max_features, embed_size, input_length=maxlen))
#     model.add(Dropout(0.5))
    model.add(GRU(8,dropout=0.3, activation='softmax'))
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

def CNN(maxlen=50, max_features=4590, embed_size=32):
    # Inputs
    comment_seq = Input(shape=[maxlen], name='x_seq')

    # Embeddings layers
    emb_comment = Embedding(max_features, embed_size)(comment_seq)

    # conv layers
    convs = []
#     filter_sizes = [2, 3, 4, 5]
    filter_sizes = [2, 3]

    for fsz in filter_sizes:
        l_conv = Conv1D(filters=10, kernel_size=fsz, activation='relu')(emb_comment)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    merge = concatenate(convs, axis=1)

    out = Dropout(0.5)(merge)
    output = Dense(16, activation='relu')(out)

    output = Dense(units=1, activation='sigmoid')(output)

    model = Model([comment_seq], output)
    #     adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

In [49]:
## read Data
input_file = "csv/training_data.csv"
dataframe = pd.read_csv(input_file, na_filter = False)
trainingData = []
label = []
for i,d in dataframe.iterrows():
    trainingData.append(d["data"])
    label.append(d["label"])


trainingData_rohe = numpy.array(trainingData)
trainingData_new = []
for td in trainingData_rohe:
    feature = (td.replace("[","").replace("]","").split(','))
    results = list(map(int, feature))
    trainingData_new.append(results)
# print(trainingData_new)
label = numpy.array(label)
trainingData_new = numpy.array(trainingData_new)
print(trainingData_new.shape)
print(label.shape)
## build model
def read_vocabulary(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        return iterable_to_dict(f.readlines())

def iterable_to_dict(arr):
    return dict((x.strip(), (i)) for (i, x) in enumerate(arr))

vocabFile = "csv/vocab.txt"
word_vocabulary = read_vocabulary(vocabFile)
CNN_model = CNN(len(trainingData_new[0]),len(vocabFile))
CNN_model.summary()
#model compile
CNN_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

(1050, 41)
(1050,)
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_seq (InputLayer)              (None, 41)           0                                            
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 41, 32)       416         x_seq[0][0]                      
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 40, 10)       650         embedding_21[0][0]               
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 39, 10)       970         embedding_21[0][0]               
_________________________________________________________________________

In [50]:
## training 
batch_size = 5
epochs = 10
es = EarlyStopping(monitor='val_loss', patience=3)
CNN_model.fit(trainingData_new, label,
          validation_split=0.25,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[es],
          shuffle=True)
i = 0
for f, b in zip(trainingData_new, label):
    i += predict(f,b)
print("total num:",len(trainingData_new))
print(i)

Train on 787 samples, validate on 263 samples
Epoch 1/10
787/787 [==============================] - 4s 5ms/step - loss: 2314254666311335424.0000 - accuracy: 0.6277 - val_loss: 991270453283256832.0000 - val_accuracy: 0.3764
Epoch 2/10
787/787 [==============================] - 2s 3ms/step - loss: 1511.8976 - accuracy: 0.7471 - val_loss: 991270387552293504.0000 - val_accuracy: 0.3764
Epoch 3/10
787/787 [==============================] - 2s 3ms/step - loss: 1511.8976 - accuracy: 0.7471 - val_loss: 991270387552293504.0000 - val_accuracy: 0.3764
Epoch 4/10
787/787 [==============================] - 2s 3ms/step - loss: 1511.8976 - accuracy: 0.7471 - val_loss: 991270387552293504.0000 - val_accuracy: 0.3764
Epoch 5/10
787/787 [==============================] - 2s 3ms/step - loss: 1511.8977 - accuracy: 0.7471 - val_loss: 991270387552293504.0000 - val_accuracy: 0.3764
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1

In [14]:
def predict(testData, truth):
    test = numpy.reshape(testData,(1,testData.shape[0]))
    pr = model.predict(test)
    
    if (pr >= 0.5 and truth == 1) or (pr< 0.5 and truth ==0):
#         print("true")
        print(pr)
        return 1
    else:
#         print("false")
        return 0

In [51]:
model = RNN(len(trainingData_new[0]),len(vocabFile))
model.summary()
#model compile
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
## training 
batch_size = 2
epochs = 10
es = EarlyStopping(monitor='val_loss', patience=3)
model.fit(trainingData_new, label,
          validation_split=0.25,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[es],
          shuffle=True)


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 41, 32)            416       
_________________________________________________________________
gru_13 (GRU)                 (None, 8)                 984       
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 9         
Total params: 1,409
Trainable params: 1,409
Non-trainable params: 0
_________________________________________________________________
Train on 787 samples, validate on 263 samples
Epoch 1/10
787/787 [==============================] - 48s 60ms/step - loss: 0.5812 - accuracy: 0.7471 - val_loss: 0.7038 - val_accuracy: 0.6008
Epoch 2/10
787/787 [==============================] - 45s 58ms/step - loss: 0.5671 - accuracy: 0.7471 - val_loss: 0.7137 - val_accuracy: 0.6008
Epoch 3/10
787/787 [===========================

In [52]:
i = 0
for f, b in zip(trainingData_new, label):
    i += predict(f,b)
print("total num:",len(trainingData_new))
print(i)

[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.255982

[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.25598285]]
[[0.255982